<a href="https://colab.research.google.com/github/abdulkadirdemirci/AssociationRuleLearning_case_1/blob/master/arl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import mlxtend
import openpyxl
import datetime
from mlxtend.frequent_patterns import apriori,association_rules

# ayarlamalar

In [2]:

pd.set_option("display.expand_frame_repr",False)
pd.set_option("display.max_columns",None)

# İŞ PROBLEMİ

Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.
Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca
ulaşılmasını sağlamaktadır.
Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak
Association Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

# VERİ SETİ HİKAYESİ

Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır.
Alınan her hizmetin tarih ve saat bilgisini içermektedir.
* UserId      : muşteri nuamrası

* serviceId   : Her kategoriye ait anonimleştirilmiş servislerdir. 
            (Örnek : Temizlik kategorisi altında koltuk yıkama servisi) 
            Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.(Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
            
* categoryId  : Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi) 

* createdate  :Hizmetin satın alındığı tarih

#################################################
# GÖREV 1 ADIM 1
* todo: armut_data.csv dosyasını okutunuz.

#################################################

In [3]:
df_ = pd.read_csv("/content/drive/MyDrive/veriler/armut_data.csv")
df = df_.copy()


In [4]:
df.head()


,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [6]:
df["CreateDate"] = pd.to_datetime(df["CreateDate"])


In [7]:
df.shape


(162523, 4)

In [8]:
df.nunique()


UserId         24826
ServiceId         50
CategoryId        12
CreateDate    117510
dtype: int64

In [9]:
df.isnull().sum()

UserId        0
ServiceId     0
CategoryId    0
CreateDate    0
dtype: int64

#################################################
# GÖREV 1 ADIM 2
* todo: ServisID her bir CategoryID özelinde farklı
 bir hizmeti temsil etmektedir.
 ServiceID ve CategoryID’yi "_" ile birleştirerek bu hizmetleri
 temsil edecek yeni bir değişken oluşturunuz.
 
#################################################

In [10]:
df["stockcode"] = df.apply(lambda row: "%s_%s" % (row["ServiceId"],row["CategoryId"]),axis=1)


In [11]:
df.head()


,UserId,ServiceId,CategoryId,CreateDate,stockcode
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [12]:
df.nunique()

UserId         24826
ServiceId         50
CategoryId        12
CreateDate    117510
stockcode         50
dtype: int64

#################################################
# GÖREV 1 ADIM 3
* todo: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır,
 herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.
 Association Rule Learning uygulayabilmek için bir sepet (fatura vb.)
 tanımı oluşturulması gerekmektedir. Burada sepet tanımı her bir
 müşterinin aylık aldığı hizmetlerdir.
 hizmetleri bir sepeti; 2017’in 10.ayında aldığı 9_4, 38_4 hizmetleri
 başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile
 tanımlanması gerekmektedir. Bunun için öncelikle sadece yıl ve ay
 içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz
 date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayınız.
 
#################################################

In [13]:
df["year"] = df["CreateDate"].dt.year


In [14]:
df["month"] = df["CreateDate"].dt.month

In [16]:
df["transactions"] = df.apply(lambda row: "%s_%s_%s" % (row["UserId"],row["year"],row["month"]),axis=1)

#################################################
# GÖREV 2 ADIM 1
* todo: sepet, hizmet pivot table’i oluşturunuz.

#################################################

In [17]:
pivot_df = df.groupby(["transactions","stockcode"])["stockcode"].\
    count().\
    unstack().\
    fillna(0).\
    applymap(lambda x: 1 if x > 0 else 0)

#################################################
# GÖREV 2 ADIM 2
* todo: Birliktelik kurallarını oluşturunuz.

apriori' ye uygun hale gelen verisetini kullanarak support degerlerini belirlemek
#################################################

In [18]:
pivot_df_apriori = apriori(pivot_df,min_support=0.01,use_colnames=True)
pivot_df_apriori.head()

,support,itemsets
0,0.019728,(0_8)
1,0.026523,(11_11)
2,0.029374,(12_7)
3,0.056627,(13_11)
4,0.023406,(14_7)


In [19]:
rules = association_rules(pivot_df_apriori,metric="support",min_threshold=0.01).sort_values("lift",ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(22_0),(25_0),0.047515,0.042895,0.011120,0.234043,5.456141,0.009082,1.249553
11,(25_0),(22_0),0.042895,0.047515,0.011120,0.259247,5.456141,0.009082,1.285834
18,(38_4),(9_4),0.066568,0.041393,0.010067,0.151234,3.653623,0.007312,1.129413
19,(9_4),(38_4),0.041393,0.066568,0.010067,0.243216,3.653623,0.007312,1.233418
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262


#################################################
# GÖREV 2 ADIM 3
* todo: arl_recommender fonksiyonunu kullanarak en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.

#################################################

In [20]:
def arl_recomender(df,product_id,rec_lim=1):
    recomendation_list = []
    product_id = product_id
    for index,antecedent in enumerate(rules["antecedents"]):
         for i in list(antecedent):
            if i == product_id:
                recomendation_list.append(list(rules.iloc[index]["consequents"])[0])
    return  recomendation_list[:rec_lim]


In [21]:
arl_recomender(rules,"15_1")

['33_4']